<a href="https://colab.research.google.com/github/chun-ming-Lin/yolov4_mask/blob/master/colab/yolo_convert_keras_h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 將模型編譯成.pb檔

In [ ]:
!nvidia-smi

Fri Aug 13 08:17:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone 'https://github.com/hunglc007/tensorflow-yolov4-tflite' 'tf_yolov4'

Cloning into 'tf_yolov4'...
remote: Enumerating objects: 780, done.
remote: Total 780 (delta 0), reused 0 (delta 0), pack-reused 780
Receiving objects: 100% (780/780), 197.64 MiB | 24.59 MiB/s, done.
Resolving deltas: 100% (370/370), done.


In [ ]:
!cp '/content/drive/MyDrive/YOLOv4/YOLOv4_mask/backup/yolov4-mask_best.weights' 'yolov4-mask_last.weights'

In [ ]:
!cp '/content/drive/MyDrive/YOLOv4/YOLOv4_mask/obj.names' 'obj.names'
!cp '/content/drive/MyDrive/YOLOv4/YOLOv4_mask/obj.data' 'obj.data'

In [ ]:
import os
os.chdir('tf_yolov4')

更改 core/config.py 中的 __C.YOLO.CLASSES 

In [ ]:
# Save weights and model
!python save_model.py --weights /content/yolov4-mask_last.weights --output /content/yolov4 --input_size 416 --model yolov4

2021-08-13 08:19:47.206920: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-13 08:19:48.936064: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-13 08:19:48.957233: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-13 08:19:48.958112: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2021-08-13 08:19:48.958161: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-13 08:19:48.963144: I tensorflow/stream_executor/platform/defa

In [ ]:
os.chdir('/content')

# 載入模型

In [ ]:
# # .pb檔載入
# import tensorflow as tf
# model = tf.keras.models.load_model('/content/yolov4')
# # save to .h5 file
# model.save('yolov4_mask.h5')

In [ ]:
!cp '/content/yolov4_mask.h5' '/content/drive/MyDrive/YOLOv4/YOLOv4_mask/yolov4_mask.h5'

In [ ]:
import tensorflow as tf
# model = tf.keras.models.load_model('/content/yolov4')
model = tf.keras.models.load_model('/content/yolov4_mask.h5')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

classes = [line.strip() for line in open('/content/obj.names')]
img_original = cv2.imread('/content/yasin.jpg')
img = cv2.resize(img_original, (416, 416))
img = img / 255
img = img.reshape(1, 416, 416, 3)

height, width, channels = img_original.shape

In [ ]:
preds = model.predict(img)

In [ ]:
preds.shape

(1, 2, 7)

In [ ]:
for pred in preds:
  boxes = pred[:, 0:4]
  pred_conf = pred[:, 4:]


In [ ]:
for conf in pred_conf:
  print(np.argmax(conf))

1
0


In [ ]:
for box in boxes:
  y1 = int(box[0] * height)
  x1 = int(box[1] * width)
  y2 = int(box[2] * height)
  x2 = int(box[3] * width)
  pt1 = (x1, y1)
  pt2 = (x2, y2)
  color = (0, 200, 0)
  cv2.rectangle(img_original, pt1, pt2, color, thickness=1)
cv2_imshow(img_original)

# 問題
在編譯成tensorflow的樣子時，出現以下問題
```
# Q:
ValueError: cannot reshape array of size 4565322 into shape (1024,512,3,3)
# A:
  到core資料夾 -> config.py -> __C.YOLO.CLASSES 換成你的obj.names
```
